In [1]:
from PIL import Image

In [2]:
import numpy as np

In [3]:
from scipy import ndimage

In [4]:
import pandas as pd

In [5]:
meta_data = pd.read_csv("/home/philipp/data/saez/cellData.csv")
meta_data

,SampleID,cellLabelInImage,cellSize,C,Na,Si,P,Ca,Fe,dsDNA,...,MPO,Keratin6,HLA_Class_1,Ta,Au,tumorYN,tumorCluster,Group,immuneCluster,immuneGroup
0,1,2,146,0,-0.59061,0.875220,-2.57650,-0.806060,-0.234400,-1.308000,...,-0.13903,-0.41883,-0.53738,2.48110,2.326600,1,0,6,0,0
1,1,3,102,0,-0.49870,0.017464,-1.22490,-0.501520,-1.341200,0.522570,...,-0.13903,-0.52769,-0.62060,2.30430,2.187100,0,0,2,46,4
2,1,4,43,0,-1.48730,-0.630440,-1.91070,-1.228000,-1.393700,-1.782200,...,-0.13903,0.18774,-1.75050,1.39330,1.478200,1,0,6,0,0
3,1,5,211,0,-1.00530,-0.532270,-1.74300,-0.944850,-1.099600,-0.057906,...,-0.13903,-0.30976,-0.52445,1.88680,1.586200,1,0,6,0,0
4,1,6,177,0,0.15803,-0.710290,0.51737,-0.096251,-1.052600,0.355020,...,-0.13903,-0.52769,-0.99773,-0.51027,-0.496270,0,0,2,75,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201651,44,1402,198,0,3.79060,1.212600,0.82351,1.194900,0.617350,1.530700,...,-0.13903,-0.34334,-1.74170,-0.60616,-0.049150,1,0,6,0,0
201652,44,1403,111,0,3.89980,0.119160,1.03970,1.089000,-0.075447,1.428700,...,-0.13903,-0.52769,-1.77350,-0.54045,-0.260810,1,1,1,0,0
201653,44,1404,100,0,3.54820,-0.578650,-2.12650,0.803610,-0.788880,0.566460,...,-0.13903,-0.52769,-1.85290,-0.73996,0.055092,1,0,6,0,0
201654,44,1405,129,0,3.59890,2.929000,-1.77210,1.375700,-0.301130,-1.198100,...,-0.13903,-0.52769,-1.85290,0.56260,1.092300,1,0,6,0,0


In [6]:
cells_per_sample = meta_data.groupby(["SampleID"])["SampleID"].agg(['count'])
cells_per_sample

,count
SampleID,
1,5167
2,3028
3,6315
4,6643
5,5406
6,5998
7,3410
8,3136
9,6139


In [7]:
dict_cells = cells_per_sample.to_dict()["count"]

In [8]:
patient_ids = np.arange(1, 42)[np.arange(1,42) != 30]
patient_ids

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41])

In [9]:
df = pd.DataFrame()
for id in patient_ids:
    # Read in matrix
    im = Image.open(f"/home/philipp/data/saez/p{id}_labeledcellData.tiff")
    imarray = np.array(im)
    # Mask (binarize for each label)
    lbl = ndimage.label(imarray)[0]
    # Find center of mass for each label
    coordinates = ndimage.measurements.center_of_mass(imarray, lbl, np.arange(1, dict_cells[id]+1))
    df_tmp = pd.DataFrame({"id": id, "coordinate": coordinates})
    df = df.append(df_tmp)

In [10]:
df

,id,coordinate
0,1,"(1010.4863647278729, 1045.208291815486)"
1,1,"(36.32839506172839, 161.80987654320987)"
2,1,"(37.88076923076923, 190.15769230769232)"
3,1,"(35.029411764705884, 203.74117647058824)"
4,1,"(34.443636363636365, 224.22545454545454)"
...,...,...
4527,41,"(1640.0, 247.5631067961165)"
4528,41,"(1644.8415584415584, 951.4064935064935)"
4529,41,"(1630.6666666666667, 1545.3333333333333)"
4530,41,"(1645.3748308525035, 1630.345060893099)"


In [11]:
df.to_csv("/home/philipp/Saez/eda/coordinates.csv")